In [1]:
""" Cilj ovog zadatka je primijeniti algoritam kolonije mrava 
(Ant Colony Optimization - ACO) za rješavanje problema optimizacije 
rute dostave paketa. U ovom kontekstu, cilj je pronaći najkraću moguću 
rutu koja će omogućiti dostavljaču da posjeti sve zadane adrese u što 
kraćem vremenu ili s najmanjim troškom. S obzirom na to da svaka dostava
može biti povezana s različitim troškovima (kao što su udaljenost, vrijeme ili gorivo)
 """

"""Kako bi se riješio ovaj problem, ACO koristi skup mrava koji simultano istražuju 
različite rute između adresa. Na temelju odluka donesenih od strane mrava, koji 
biraju sljedeće adrese uzimajući u obzir koncentraciju feromona i udaljenost, 
algoritam s vremenom poboljšava rješenja. Feromoni deponirani na kraćim rutama 
signaliziraju da su te rute bolje, pa tako drugi mravi imaju veću vjerojatnost 
da će ih odabrati. Kroz ponavljanje ovog procesa u više iteracija, algoritam se 
fokusira na rute koje daju najbolji rezultat, minimizirajući ukupnu udaljenost i 
trošak dostave, čime se postiže optimalan raspored za dostavu paketa. """



'Kako bi se riješio ovaj problem, ACO koristi skup mrava koji simultano istražuju \nrazličite rute između adresa. Na temelju odluka donesenih od strane mrava, koji \nbiraju sljedeće adrese uzimajući u obzir koncentraciju feromona i udaljenost, \nalgoritam s vremenom poboljšava rješenja. Feromoni deponirani na kraćim rutama \nsignaliziraju da su te rute bolje, pa tako drugi mravi imaju veću vjerojatnost \nda će ih odabrati. Kroz ponavljanje ovog procesa u više iteracija, algoritam se \nfokusira na rute koje daju najbolji rezultat, minimizirajući ukupnu udaljenost i \ntrošak dostave, čime se postiže optimalan raspored za dostavu paketa. '

In [2]:
import random
import numpy as np

In [3]:

# Parametri algoritma
num_ants = 20           # Broj mrava
num_iterations = 100    # Broj iteracija
alpha = 1               # Utjecaj feromona na izbor sljedeće adrese
beta = 2                # Utjecaj udaljenosti na izbor sljedeće adrese
rho = 0.5               # Brzina isparavanja feromona
q = 100                 # Količina feromona koju mravi deponiraju

# Inicijalizacija adresa (npr. 5 lokacija u gradu)
addresses = np.array([[0, 0], [1, 2], [2, 4], [4, 4], [5, 1]])

In [4]:
# Funkcija za izračun udaljenosti između dviju adresa
def calculate_distance(address1, address2):
    return np.linalg.norm(address1 - address2)

In [5]:
# Kreiranje matrice udaljenosti između adresa
distances = np.zeros((len(addresses), len(addresses)))
for i in range(len(addresses)):
    for j in range(len(addresses)):
        if i != j:
            distances[i][j] = calculate_distance(addresses[i], addresses[j])


In [6]:
# Inicijalizacija matrice feromona (početno postavljamo feromone na malu vrijednost)
pheromones = np.ones_like(distances) * 0.1

In [7]:
# Funkcija za odabir sljedeće adrese temeljem feromona i udaljenosti
def select_next_address(current_address, visited_addresses):
    probabilities = []
    total_pheromone = 0

    # Računanje vjerovatnoće za svaku adresu temeljem feromona i udaljenosti
    for i in range(len(addresses)):
        if i not in visited_addresses:
            pheromone = pheromones[current_address][i] ** alpha    # Feromon na adresi
            distance = distances[current_address][i] ** (-beta)    # Inverzna udaljenost
            probability = pheromone * distance
            probabilities.append(probability)
            total_pheromone += probability
        else:
            probabilities.append(0)

    # Normalizacija vjerovatnoća
    probabilities = [p / total_pheromone for p in probabilities]
    # Odabir sljedeće adrese temeljem normaliziranih vjerovatnoća
    next_address = np.random.choice(range(len(addresses)), p=probabilities)
    return next_address


In [8]:
#algoritam kolonije mrava za optimizaciju rute dostave
def ant_colony_optimization():
    global pheromones  # Deklaracija feromona kao globalne varijable
    best_route = None
    best_distance = float('inf')

    # Iteracije koje simuliraju putovanja mrava
    for iteration in range(num_iterations):
        all_routes = []
        all_distances = []

        # Svaki mrav kreće na svoje putovanje
        for ant in range(num_ants):
            route = [random.randint(0, len(addresses)-1)]  # Početna adresa (random)
            visited_addresses = set(route)

            # Putovanje mrava - odabir sljedeće adrese dok ne posjeti sve adrese
            for _ in range(len(addresses) - 1):
                current_address = route[-1]
                next_address = select_next_address(current_address, visited_addresses)
                route.append(next_address)
                visited_addresses.add(next_address)

            # Izračun ukupne udaljenosti rute mrava
            total_distance = sum(distances[route[i], route[i+1]] for i in range(len(route) - 1))
            all_routes.append(route)
            all_distances.append(total_distance)

            # Ažuriranje najbolje rute (ako je trenutna ruta bolja)
            if total_distance < best_distance:
                best_distance = total_distance
                best_route = route

        # Isparavanje feromona (smanjenje svih vrijednosti feromona)
        pheromones *= (1 - rho)

        # Deponovanje feromona na staze koje su mravi prošli
        for i in range(num_ants):
            for j in range(len(all_routes[i]) - 1):
                pheromones[all_routes[i][j], all_routes[i][j + 1]] += q / all_distances[i]

    return best_route, best_distance

In [9]:
# Pokretanje algoritma kolonije mrava
best_route, best_distance = ant_colony_optimization()

In [10]:
# Prikaz najboljeg puta i njegove udaljenosti
print(f"Najbolja ruta: {best_route}")
print(f"Ukupna udaljenost: {best_distance}")

Najbolja ruta: [0, np.int64(1), np.int64(2), np.int64(3), np.int64(4)]
Ukupna udaljenost: 9.63441361516796
